In [ ]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import os

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults

from langsmith import traceable
from langsmith.wrappers import wrap_openai


load_dotenv(find_dotenv(), override=True)

True

## QuickStart

In [66]:
llm = ChatOpenAI(model_name = "gpt-4o")
messages = [
    SystemMessage(content="You are an expert fund manager with a deep understanding of financial instruments and investment strategies."),
    HumanMessage(content="What is US Treasury Bonds? and whats the difference with bills?"),
]

response = llm.invoke(messages)
print(response.content)

U.S. Treasury securities are debt instruments issued by the Department of the Treasury to finance the federal government's operations. They are considered some of the safest investments because they are backed by the full faith and credit of the United States government. There are several types of Treasury securities, including Treasury bonds and Treasury bills, each with distinct characteristics.

### Treasury Bonds

- **Maturity**: Treasury bonds have the longest maturity among Treasury securities, typically ranging from 20 to 30 years.
- **Interest Payments**: They pay a fixed interest rate, also known as a coupon, every six months until maturity.
- **Purpose**: They are often used by investors seeking a long-term, fixed-income investment.
- **Risk**: While considered very safe, they carry interest rate risk, meaning their market value can fluctuate with changes in interest rates.

### Treasury Bills

- **Maturity**: Treasury bills, or T-bills, are short-term securities with maturit

In [57]:
template = "Explain the concept of {concept} in simply way"
prompt = PromptTemplate(
    input_variables=["concept"],
    template=template
)

prompt_messages = prompt.format(concept="US Treasury Bonds")
print(prompt_messages)

Explain the concept of US Treasury Bonds in simply way


In [60]:
def chat(message):
    llm = ChatOpenAI(model_name="gpt-4o-mini")
    messages = [
        SystemMessage(content="You are an expert fund manager with a deep understanding of financial instruments and investment strategies."),
        HumanMessage(content=message),
    ]

    response = llm.invoke(messages)
    return response.content   

In [61]:
chat(prompt_messages)

'Sure! U.S. Treasury Bonds are basically loans that you can give to the U.S. government. Here’s how it works in simple terms:\n\n1. **Borrowing Money**: When the government needs money to pay for things like roads, schools, or military, it borrows money by issuing Treasury Bonds.\n\n2. **Buying Bonds**: When you buy a Treasury Bond, you are lending money to the government. In return, the government promises to pay you back the amount you lent (the principal) plus interest.\n\n3. **Interest Payments**: Treasury Bonds pay interest, called the coupon rate, every six months until the bond matures. This means you get regular payments as a reward for lending your money.\n\n4. **Maturity**: Treasury Bonds usually have a long-term maturity, typically ranging from 10 to 30 years. This means the government will return your principal back to you at the end of that period.\n\n5. **Safety**: U.S. Treasury Bonds are considered very safe investments because they are backed by the "full faith and cred

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke("US Treasury Bonds")  # Example usage of the chain

{'concept': 'US Treasury Bonds',
 'text': 'US Treasury Bonds are a type of long-term government debt issued by the United States Department of the Treasury. They are essentially loans that investors give to the US government in exchange for periodic interest payments and the return of the bond\'s face value when it matures. Here\'s a simple breakdown:\n\n1. **Issuer**: The US government issues these bonds to raise money for various expenses, including infrastructure, public services, and paying off existing debt.\n\n2. **Maturity**: Treasury Bonds typically have a long maturity period, often 20 to 30 years. This means the government agrees to pay back the full amount borrowed (the principal) after this period.\n\n3. **Interest Payments**: Investors receive interest payments, known as coupon payments, every six months. The interest rate is fixed when the bond is issued.\n\n4. **Safety**: These bonds are considered one of the safest investments because they are backed by the "full faith 

In [73]:
second_prompt = PromptTemplate(
    input_variables=["concept"],
    template="Explain the concept of {concept} like im 5"
)

chain_2 = LLMChain(llm=llm, prompt=second_prompt)

In [74]:
all_chain = SimpleSequentialChain(chains = [chain, chain_2], verbose=True)
all_chain.invoke("US Treasury Bonds")  # Example usage of the sequential chain



> Entering new SimpleSequentialChain chain...
U.S. Treasury Bonds are essentially loans that you give to the U.S. government. When you purchase a Treasury Bond, you are lending the government money. In return, the government promises to pay you back the full amount of the loan (the principal) on a specific date in the future, which is usually 20 or 30 years from when the bond is issued.

While you hold the bond, the government also pays you interest, usually every six months. This interest is a way to reward you for lending them your money. The rate of this interest is fixed when you buy the bond, meaning it won't change over the life of the bond.

Treasury Bonds are considered very safe investments because they are backed by the "full faith and credit" of the U.S. government, which has a long history of repaying its debts. Because of their safety, the interest rates on Treasury Bonds are typically lower than those of riskier investments.
Sure! Imagine you have a piggy bank, and you 

{'input': 'US Treasury Bonds',
 'output': 'Sure! Imagine you have a piggy bank, and you decide to let a friend borrow some of your coins. In return, your friend promises to give you all your coins back after a long time, like when you\'re a much bigger kid. This is kind of like what happens with U.S. Treasury Bonds.\n\nWhen you buy a Treasury Bond, it\'s like giving your coins to the U.S. government. They promise to give all your coins back after a set time, like 20 or 30 years. While you\'re waiting, every now and then, they give you a little extra coin as a "thank you" for letting them use your coins. This extra coin is called interest, and you get it every six months.\n\nPeople like these bonds because they trust the U.S. government to give their money back. It\'s a safe way to save coins for the future, even if it doesn\'t give you as many extra coins as some other places might.'}

In [4]:
llm = ChatOllama(model="llama3.2")
messages = [
    SystemMessage(content="You are an expert fund manager with a deep understanding of financial instruments and investment strategies."),
    HumanMessage(content="What is US Treasury Bonds? and whats the difference with bills?"),
]

# response = llm.invoke(messages)
# print(response.content)

In [ ]:
topic_memory = ConversationBufferMemory(input_key="topic", memory_key="topic_memory") # We also can use same memory_key
concept_memory = ConversationBufferMemory(input_key="concept", memory_key="concept_memory")

prompt1 = PromptTemplate(
    input_variables=["topic"],
    template="Explain the concept of {topic} in simply way"
)
chain1 = LLMChain(llm=llm, prompt=prompt1, verbose=True, output_key="concept", memory=topic_memory)

prompt2 = PromptTemplate(
    input_variables=["concept"],
    template="Summarize the explanation of {concept} in a concise manner"
)
chain2 = LLMChain(llm=llm, prompt=prompt2, verbose=True, output_key="summary", memory=concept_memory)

parent_chain = SequentialChain(chains=[chain1, chain2], input_variables=["topic"], output_variables=["concept", "summary"], verbose=True)
parent_chain.invoke({"topic": "Bitcoin"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Explain the concept of Bitcoin in simply way

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Summarize the explanation of Bitcoin is a digital currency that uses cryptography (secret codes) to secure and verify transactions. Here's a simplified explanation:

**What is Bitcoin?**

Bitcoin is a type of money that exists only on computers and phones. It's like the money in your wallet, but instead of physical coins and bills, it's made of computer code.

**How does it work?**

1. **Mining**: New Bitcoins are created through a process called "mining," which involves solving complex math problems using powerful computers.
2. **Transactions**: When someone wants to send Bitcoin to another person, they create a transaction and broadcast it to the entire network of computers that use the Bitcoin system.
3. **Verification**: Special nodes on the network verify the

{'topic': 'Bitcoin',
 'concept': 'Bitcoin is a digital currency that uses cryptography (secret codes) to secure and verify transactions. Here\'s a simplified explanation:\n\n**What is Bitcoin?**\n\nBitcoin is a type of money that exists only on computers and phones. It\'s like the money in your wallet, but instead of physical coins and bills, it\'s made of computer code.\n\n**How does it work?**\n\n1. **Mining**: New Bitcoins are created through a process called "mining," which involves solving complex math problems using powerful computers.\n2. **Transactions**: When someone wants to send Bitcoin to another person, they create a transaction and broadcast it to the entire network of computers that use the Bitcoin system.\n3. **Verification**: Special nodes on the network verify the transaction to ensure it\'s legitimate and follow the rules.\n4. **Blockchain**: The verified transactions are stored in a public ledger called the blockchain, which is like a digital bookkeeping system.\n\n

In [10]:
print(topic_memory.buffer)
print(concept_memory.buffer)

Human: Bitcoin
AI: Bitcoin is a digital currency that uses cryptography (secret codes) to secure and verify transactions. Here's a simplified explanation:

**What is Bitcoin?**

Bitcoin is a type of money that exists only on computers and phones. It's like the money in your wallet, but instead of physical coins and bills, it's made of computer code.

**How does it work?**

1. **Mining**: New Bitcoins are created through a process called "mining," which involves solving complex math problems using powerful computers.
2. **Transactions**: When someone wants to send Bitcoin to another person, they create a transaction and broadcast it to the entire network of computers that use the Bitcoin system.
3. **Verification**: Special nodes on the network verify the transaction to ensure it's legitimate and follow the rules.
4. **Blockchain**: The verified transactions are stored in a public ledger called the blockchain, which is like a digital bookkeeping system.

**Key characteristics:**

1. **D

## Langsmith

### Quickstart

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{question}"),
    ]
)

In [5]:
llm = ChatOpenAI(model_name="gpt-4o-mini")
chain = prompt | llm | StrOutputParser()
chain.invoke({"question": "How do market maker somehow manipulate order bool?"})

C:\Users\asus\AppData\Local\Temp\ipykernel_31316\2753955043.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini")


"Market makers play a crucial role in providing liquidity to financial markets by being ready to buy and sell assets at any time. While they operate under a regulated framework, there are concerns and accusations regarding their potential to manipulate the order book. Here are some ways that market makers might engage in practices that could be seen as manipulative, although it’s important to note that not all market makers engage in such practices, and many operate within legal and ethical guidelines:\n\n1. **Order Spoofing**: Market makers might place large orders that they have no intention of executing to create a false impression of demand or supply. This can mislead other traders about the true market conditions, causing them to react in ways that benefit the market maker.\n\n2. **Quote Stuffing**: This involves rapidly placing and canceling orders to create confusion and latency in the market. By overwhelming the order book with orders, a market maker can manipulate the percepti

### Traceable

In [16]:
web_search_tool = TavilySearchResults(max_results=1)
openai_client = wrap_openai(OpenAI())

In [36]:
@traceable
def question_context(question: str):
    content = web_search_tool.invoke(question)
    context = "\n".join([web["content"] for web in content])
    return context

@traceable
def ask_question(question: str):
    context = question_context(question)
    
    system_message = """
    Give explanation in a simple way to curious high school student in challenging mindset based on the context below.
    Context: {context}
    
    """
    
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "{question}"),
    ])
    
    llm = ChatOpenAI(model_name="gpt-4o-mini")
    chain = prompt | llm | StrOutputParser()
    
    
    return chain.invoke({"question": question, "context": context})


In [37]:
ask_question("How do interest rate affect stock market?")

"Great question! Let’s break it down simply.\n\nInterest rates are like the cost of borrowing money. When the Federal Reserve (which is like the bank for banks) changes these rates, it can have a big impact on how people and businesses spend money, which in turn affects the stock market.\n\n1. **When Interest Rates Go Down**: If the Federal Reserve lowers interest rates, it becomes cheaper to borrow money. This means that people and businesses are more likely to take out loans to buy things or invest in new projects. As spending increases, businesses can earn more money, which often leads to higher stock prices. So, when interest rates drop, the stock market usually goes up.\n\n2. **When Interest Rates Go Up**: On the flip side, when the Federal Reserve raises interest rates, borrowing becomes more expensive. This can make people and businesses think twice about spending money. If they cut back on spending, businesses might make less money, and this can lead to lower stock prices. So, 